In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
#cell = pd.read_csv("/content/drive/MyDrive/labels_ccr-1-stem-1_csv.csv")
cell = pd.read_csv("/content/drive/MyDrive/rawimg/labels-ccr (2).csv")

In [ ]:
cell['xmax'] = cell['xmin'] + cell['delta_x']
cell['ymax'] = cell['ymin'] + cell['delta_y']


print(cell['xmin'] )
print(cell)
print(cell['label'][0])
print(cell['xmax'][0])


0      1113
1      1041
2      1165
3      1299
4       100
       ... 
785     137
786      37
787     234
788    1183
789    1342
Name: xmin, Length: 790, dtype: int64
          label  xmin  ymin  delta_x  delta_y        image_name  total_x  \
0        vessel  1113   517      249      214  Ccr-1-stem-1.png     1600   
1        vessel  1041   667      129      122  Ccr-1-stem-1.png     1600   
2        vessel  1165   186      272      198  Ccr-1-stem-1.png     1600   
3        vessel  1299    12      230      173  Ccr-1-stem-1.png     1600   
4    parenchyma   100   248      103       86  Ccr-1-stem-1.png     1600   
..          ...   ...   ...      ...      ...               ...      ...   
785  parenchyma   137   269      118      119  Rca-3-root-4.png     1600   
786  parenchyma    37   442      165      138  Rca-3-root-4.png     1600   
787  parenchyma   234   378      116      135  Rca-3-root-4.png     1600   
788  parenchyma  1183   611      114      109  Rca-3-root-4.png     16

In [ ]:
from pathlib import Path
from PIL import Image
import cv2
import os
import numpy as np
import io
from google.colab import files
from google.colab import drive



In [ ]:
import csv
import random

In [22]:
x = y = z = 0 # Making global variable
#open a file
#NOTE: be careful with overwriting or running twice, etc.
file1 = open('/content/drive/MyDrive/rawimg/sw_croppedimg_index - sw_croppedimg_index (1) - sw_croppedimg_index - sw_croppedimg_index (1).csv','w')
file2 = open('/content/drive/MyDrive/rawimg/nonaugfile.csv','w')
writer = csv.writer(file1)
writer2 = csv.writer(file2)
#print(random.randint(1,3))

def image_transform(label, xmin, xmax, ymin, ymax, image_name,species):
    global x, y, z
    # Opens a image in RGB mode
    im = Image.open(f"/content/drive/MyDrive/rawimg/{image_name}")
    
    # Setting the points for cropped image/ ask mike about cropping
    #transferlearning
    left = xmin
    top = ymin #should be ymin #was 1200 - ymax
    right = xmax
    bottom = ymax  #should be ymax #was 1200 - ymin

    # Cropped image of above dimension
    # (It will not change original image)
    im1 = im.crop((left, top, right, bottom))
    im1 = im1.resize((128,128))
    

# Shows the image in image viewer
    if label == "vessel" : 
      if(species == "Ccr"):
    #    im1.save(f"/content/drive/MyDrive/hhcroppedimg/TestIMGVes/testvessel{x}.png")
        writer2.writerow(['testvessel' + str(x) + '.png',0])
        x+=1
      else:
       # im1.save(f"/content/drive/MyDrive/hhcroppedimg/Vessels/vessel{x}.png")
        writer.writerow(['vessel' + str(x) + '.png',0])
        x+=1
    if label == "parenchyma":
      if(species == "Ccr"):
      # im1.save(f"/content/drive/MyDrive/hhcroppedimg/TestImgPar/testparenchyma{y}.png")
        writer2.writerow(['testparenchyma' + str(y) + '.png',1])
        y+=1
      else:
       # im1.save(f"/content/drive/MyDrive/hhcroppedimg/Parenchyma/parenchyma{y}.png")
        writer.writerow(['parenchyma' + str(y) + '.png',1])
     
        y+=1
    if label == "fiber":
      if(species == "Ccr"):
       # im1.save(f"/content/drive/MyDrive/hhcroppedimg/TestImgFib/testfiber{z}.png")
        writer2.writerow(['testfiber' + str(z) + '.png',1])
        z+=1
      else:
       # im1.save(f"/content/drive/MyDrive/hhcroppedimg/Fiber/fiber{z}.png")
        writer.writerow(['fiber' + str(z) + '.png',1])
        z+=1

In [ ]:
#this is a for loop to crop all the images at once
#image_transform is the function we defined above
#range is a function. 
#It needs to be the same number (in this case 74) as the number of lines in our csv file.
#in brackets, put the label, then put i in brackets (for i... etc)
for i in range(790):
  image_transform(cell['label'][i],cell['xmin'][i],cell['xmax'][i],cell['ymin'][i],cell['ymax'][i],cell['image_name'][i],cell['species'][i])

file1.close()

data augmentation using keras

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

datagen = ImageDataGenerator(
        rotation_range=90,     #Random rotation between 0 and 45
        #width_shift_range=0.2,   #% shift
       # height_shift_range=0.2,
       # shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,)
        #fill_mode='reflect', cval=125)    #Also try nearest, constant, reflect, wrap


for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/Vessels'):
  x = io.imread(os.path.join('/content/drive/MyDrive/hhcroppedimg/Vessels',img))

  # Reshape the input image because ...
  #x: Input data to datagen.flow must be Numpy array of rank 4 or a tuple.
  #First element represents the number of images
  x = x.reshape((1, ) + x.shape)  #Array with shape (1, 256, 256, 3)
  i = 0
  for batch in datagen.flow(x, batch_size=16,  
                            save_to_dir='/content/drive/MyDrive/rawimg/augmentedimages/AugVessel', 
                            save_prefix='augves', 
                            save_format='png'):
      i += 1
      if i > 20:
          break  # otherwise the generator would loop indefinitely  





In [14]:
for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/Fiber'):
  x = io.imread(os.path.join('/content/drive/MyDrive/hhcroppedimg/Fiber',img))

  # Reshape the input image because ...
  #x: Input data to datagen.flow must be Numpy array of rank 4 or a tuple.
  #First element represents the number of images
  x = x.reshape((1, ) + x.shape)  #Array with shape (1, 256, 256, 3)
  i = 0
  for batch in datagen.flow(x, batch_size=16,  
                            save_to_dir='/content/drive/MyDrive/hhcroppedimg/augmentedimages/AugFiber (1)', 
                            save_prefix='augfib', 
                            save_format='png'):
      i += 1
      if i > 30:
          break  # otherwise the generator would loop indefinitely  


In [15]:
for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/Parenchyma'):
  x = io.imread(os.path.join('/content/drive/MyDrive/hhcroppedimg/Parenchyma',img))

  # Reshape the input image because ...
  #x: Input data to datagen.flow must be Numpy array of rank 4 or a tuple.
  #First element represents the number of images
  x = x.reshape((1, ) + x.shape)  #Array with shape (1, 256, 256, 3)
  i = 0
  for batch in datagen.flow(x, batch_size=16,  
                            save_to_dir='/content/drive/MyDrive/rawimg/augmentedimages/AugParenchyma', 
                            save_prefix='augpar', 
                            save_format='png'):
      i += 1
      if i > 30:
          break  # otherwise the generator would loop indefinitely  



write csv

In [24]:
import csv
import os

file3 = open('/content/drive/MyDrive/rawimg/finalseanwucsv - sw_croppedimg_index - sw_croppedimg_index (1) - sw_croppedimg_index - sw_croppedimg_index (1).csv','a')
writer5 = csv.writer(file3)



In [26]:
#for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/augmentedimages/AugFiber'):
 # writer.writerow([str(img),2])

for img in os.listdir('/content/drive/MyDrive/rawimg/augmentedimages/AugParenchyma'):
  writer5.writerow([str(img),1])
for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/augmentedimages/AugFiber (1)'):
  writer5.writerow([str(img),1])
for img in os.listdir('/content/drive/MyDrive/rawimg/augmentedimages/AugVessel'):
  writer5.writerow([str(img),0])

In [ ]:
#import csv
#import os
#file1 = open('/content/drive/MyDrive/rawimg/sw_croppedimg_index.csv','a')
#writer = csv.writer(file1)

#for img in os.listdir('/content/drive/MyDrive/rawimg/augmentedimages/AugVessel'):
 # writer.writerow([str(img),0])


In [ ]:
import csv
import os

file5 = open('/content/drive/MyDrive/rawimg/nonaugfile.csv','w')
writer3 = csv.writer(file5)

for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/TestImgPar'):
  writer3.writerow([str(img),1])
for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/TestImgFib'):
  writer3.writerow([str(img),2])
for img in os.listdir('/content/drive/MyDrive/hhcroppedimg/TestIMGVes'):
  writer3.writerow([str(img),0])
